# Transduction from observation to prediction

### Introduction

Excerpt from https://en.wikipedia.org/wiki/Transduction_(machine_learning)):

In logic, statistical inference, and supervised learning, transduction or transductive inference is reasoning from observed, specific (training) cases to specific (test) cases. In contrast, induction is reasoning from observed training cases to general rules, which are then applied to the test cases. The distinction is most interesting in cases where the predictions of the transductive model are not achievable by any inductive model. Note that this is caused by transductive inference on different test sets producing mutually inconsistent predictions.

In [1]:
# Run the observation_prediction script
# It requires Python3, tensorflow and tensor2tensor.

! ./observation_prediction.sh

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Importing user module transduction from path /home/jdroo/github.com/josd/eye
[2018-04-23 22:24:42,581] Importing user module transduction from path /home/jdroo/github.com/josd/eye
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction
[2018-04-23 22:24:42,582] Generating problems:
    observation:
      * observation_prediction
INFO:tensorflow:Generating data for observation_prediction.
[2018-04-23 22:24:42,582] Generating data for observation_prediction.
INFO:tensorflow:Generating vocab file: /home/jdroo/t2t_data/vocab.observation_prediction.16384.subwords
[2018-04-23 2

INFO:tensorflow:Reading data files from /home/jdroo/t2t_data/observation_prediction-train*
[2018-04-23 22:24:52,730] Reading data files from /home/jdroo/t2t_data/observation_prediction-train*
INFO:tensorflow:partition: 0 num_data_files: 5
[2018-04-23 22:24:52,731] partition: 0 num_data_files: 5
INFO:tensorflow:Calling model_fn.
[2018-04-23 22:24:52,873] Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
[2018-04-23 22:24:55,699] Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
[2018-04-23 22:24:55,699] Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_754_256.bottom
[2018-04-23 22:24:55,732] Transforming feature 'inputs' with symbol_modality_754_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
[2018-04-23 22:24:55,902] Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building mod

INFO:tensorflow:Running local_init_op.
[2018-04-23 22:26:08,875] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-23 22:26:08,921] Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
[2018-04-23 22:26:11,917] Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
[2018-04-23 22:26:14,021] Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
[2018-04-23 22:26:16,150] Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
[2018-04-23 22:26:17,803] Evaluation [40/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-20:26:17
[2018-04-23 22:26:17,867] Finished evaluation at 2018-04-23-20:26:17
INFO:tensorflow:Saving dict for global step 100: global_step = 100, loss = 3.8422804, metrics-observation_prediction/accuracy = 0.096087605, metrics-observation_prediction/accuracy_per_sequence = 0.0, metrics-observation_prediction/accuracy_top5 = 0.4998028, metrics-observation_prediction/approx_bleu_score = 0.58326644, metrics-observation_prediction/neg_l

INFO:tensorflow:Running local_init_op.
[2018-04-23 22:27:33,079] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-23 22:27:33,127] Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
[2018-04-23 22:27:36,159] Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
[2018-04-23 22:27:38,222] Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
[2018-04-23 22:27:40,344] Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
[2018-04-23 22:27:42,027] Evaluation [40/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-20:27:42
[2018-04-23 22:27:42,089] Finished evaluation at 2018-04-23-20:27:42
INFO:tensorflow:Saving dict for global step 200: global_step = 200, loss = 1.1841419, metrics-observation_prediction/accuracy = 0.776523, metrics-observation_prediction/accuracy_per_sequence = 0.0, metrics-observation_prediction/accuracy_top5 = 0.89369756, metrics-observation_prediction/approx_bleu_score = 0.21239558, metrics-observation_prediction/neg_log

INFO:tensorflow:Running local_init_op.
[2018-04-23 22:28:56,979] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-23 22:28:57,027] Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
[2018-04-23 22:29:00,068] Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
[2018-04-23 22:29:02,181] Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
[2018-04-23 22:29:04,289] Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
[2018-04-23 22:29:06,027] Evaluation [40/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-20:29:06
[2018-04-23 22:29:06,090] Finished evaluation at 2018-04-23-20:29:06
INFO:tensorflow:Saving dict for global step 300: global_step = 300, loss = 0.59426934, metrics-observation_prediction/accuracy = 0.89875895, metrics-observation_prediction/accuracy_per_sequence = 0.2368, metrics-observation_prediction/accuracy_top5 = 0.94749296, metrics-observation_prediction/approx_bleu_score = 0.6052667, metrics-observation_prediction/ne

...

INFO:tensorflow:Running local_init_op.
[2018-04-23 22:48:28,384] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-23 22:48:28,430] Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
[2018-04-23 22:48:31,461] Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
[2018-04-23 22:48:33,561] Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
[2018-04-23 22:48:35,705] Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
[2018-04-23 22:48:37,381] Evaluation [40/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-20:48:37
[2018-04-23 22:48:37,445] Finished evaluation at 2018-04-23-20:48:37
INFO:tensorflow:Saving dict for global step 1700: global_step = 1700, loss = 0.061195187, metrics-observation_prediction/accuracy = 0.9895091, metrics-observation_prediction/accuracy_per_sequence = 0.9239, metrics-observation_prediction/accuracy_top5 = 0.99902713, metrics-observation_prediction/approx_bleu_score = 0.84668106, metrics-observation_prediction

INFO:tensorflow:Running local_init_op.
[2018-04-23 22:49:52,230] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-23 22:49:52,277] Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
[2018-04-23 22:49:55,287] Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
[2018-04-23 22:49:57,359] Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
[2018-04-23 22:49:59,469] Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
[2018-04-23 22:50:01,141] Evaluation [40/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-20:50:01
[2018-04-23 22:50:01,206] Finished evaluation at 2018-04-23-20:50:01
INFO:tensorflow:Saving dict for global step 1800: global_step = 1800, loss = 0.05993673, metrics-observation_prediction/accuracy = 0.98999554, metrics-observation_prediction/accuracy_per_sequence = 0.9269, metrics-observation_prediction/accuracy_top5 = 0.9990666, metrics-observation_prediction/approx_bleu_score = 0.848146, metrics-observation_prediction/ne

INFO:tensorflow:Running local_init_op.
[2018-04-23 22:51:15,667] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-23 22:51:15,714] Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
[2018-04-23 22:51:18,737] Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
[2018-04-23 22:51:20,795] Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
[2018-04-23 22:51:22,938] Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
[2018-04-23 22:51:24,610] Evaluation [40/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-20:51:24
[2018-04-23 22:51:24,674] Finished evaluation at 2018-04-23-20:51:24
INFO:tensorflow:Saving dict for global step 1900: global_step = 1900, loss = 0.05453561, metrics-observation_prediction/accuracy = 0.99282205, metrics-observation_prediction/accuracy_per_sequence = 0.948, metrics-observation_prediction/accuracy_top5 = 0.99913234, metrics-observation_prediction/approx_bleu_score = 0.8536835, metrics-observation_prediction/n

INFO:tensorflow:Running local_init_op.
[2018-04-23 22:52:39,345] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-23 22:52:39,395] Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
[2018-04-23 22:52:42,444] Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
[2018-04-23 22:52:44,527] Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
[2018-04-23 22:52:46,628] Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
[2018-04-23 22:52:48,307] Evaluation [40/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-20:52:48
[2018-04-23 22:52:48,371] Finished evaluation at 2018-04-23-20:52:48
INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.05136103, metrics-observation_prediction/accuracy = 0.99282205, metrics-observation_prediction/accuracy_per_sequence = 0.948, metrics-observation_prediction/accuracy_top5 = 0.99910605, metrics-observation_prediction/approx_bleu_score = 0.8513485, metrics-observation_prediction/n

Instructions for updating:
keep_dims is deprecated, use keepdims instead
[2018-04-23 22:53:01,233] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/beam_search.py:93: calling reduce_logsumexp (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Done calling model_fn.
[2018-04-23 22:53:01,304] Done calling model_fn.
INFO:tensorflow:Graph was finalized.
[2018-04-23 22:53:01,469] Graph was finalized.
2018-04-23 22:53:01.470062: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-04-23 22:53:01.560975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-04-23 22:53:01.561441: I tensorflow/core/

In [2]:
# See the transductions

! pygmentize -g sample.observation
print("->-")
! pygmentize -g sample.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N
A_TURBINE producing 9533 kW
